### Kriegres Usage Examples

**Importing Kriegres**
- Dependencies like `Pyscopg2` and `Pandas` are imported automatically with Kriegres

In [2]:
# importing the class
import kriegres

**Instantiating the Kriegres Class**  
- Instantiation requires a host and a port

In [3]:
my_db = kriegres.kriegres(host = "localhost",port = "5400",)

**Kriegres Attributes**  
- `.host` : Lists current instantiations host.
- `.port` : Lists current instantiations port.
- `.database` : Lists the current connected database.
- `.server_connection` : Lists both the host and the port.
- `.db_connection` : Lists the host, port and current database.


_ A database is not required for class instantiation and will thus listing the database attribute prior to connecting to a specific database will return `None`._

In [4]:
# Check the Attributes
print(my_db.server_connection)
print(my_db.host)
print(my_db.port)
print(my_db.database)
print(my_db.db_connection)

host='localhost' port='5400'
localhost
5400
None
None


**Set the database**  
Sets the active database to one on the connected server, granted passed database exists.

In [5]:
my_db.set_database('dsi')

In [21]:
print(my_db.server_connection)
print(my_db.host)
print(my_db.port)
print(my_db.database)
print(my_db.db_connection)

host='localhost' port='5400'
localhost
5400
dsi
host='localhost' dbname='dsi' port='5400'


**List current databases on the connected server**

In [6]:
my_db.get_databases()

['postgres',
 'samstack',
 'template1',
 'template0',
 'suppliers',
 'sam',
 'dsi',
 'test',
 'd3']

**Create a new database on the connected server**

In [9]:
my_db.create_database('D3')
my_db.get_databases()

['postgres',
 'samstack',
 'template1',
 'template0',
 'suppliers',
 'sam',
 'dsi',
 'test',
 'd3']

**Delete a database on the connected server**
- **Warning** : This currently has no confirmation measure and will instantly delete the passed database.

In [10]:
my_db.delete_database('D3')
my_db.get_databases()

['postgres',
 'samstack',
 'template1',
 'template0',
 'suppliers',
 'sam',
 'dsi',
 'test']

**Get the Tables currently on each Database**  
- Returns a dictionary where keys are Database names and values are lists of public tables.
- This only returns publicly viewable tables.
- If a database is not publicly viewable, a value of `None` will be returned as a value.

In [11]:
my_db.database_schemas()

{'dsi': ['iris', 'products'],
 'postgres': None,
 'sam': ['iris', 'my_table_name', 'products'],
 'samstack': None,
 'suppliers': ['products'],
 'template0': None,
 'template1': None,
 'test': ['iris']}

**Returning information from a table**
- Unlike previous commands, this takes a valid SQL query.
- Returns a Pandas style dataframe of the query result.

_Has not been tested with queries where tables are joined._

_ The Table `"iris"` was an existing table on this database that I had been using to test the class during construction. _

In [13]:
iris_table = my_db.query('''SELECT * from iris''')
iris_table.head(10)

,index,sepal_length,sepal_width,petal_length,petal_width,target
0,0,5.1,3.5,1.4,0.2,0
1,1,4.9,3.0,1.4,0.2,0
2,2,4.7,3.2,1.3,0.2,0
3,3,4.6,3.1,1.5,0.2,0
4,4,5.0,3.6,1.4,0.2,0
5,5,5.4,3.9,1.7,0.4,0
6,6,4.6,3.4,1.4,0.3,0
7,7,5.0,3.4,1.5,0.2,0
8,8,4.4,2.9,1.4,0.2,0
9,9,4.9,3.1,1.5,0.1,0


**Insert a list of values into a table**  
- If the value type in the list does not match that in the associated row in the table, an error will return.
- This method only accepts a single list whose length is equal to that of the number of columns in the target table, passing a longer or shorter list will return a custom error message.  

In [14]:
my_db.insert_one('iris', [11,22,33,44,55,66],)

my_db.query('''SELECT * from iris''').tail(10)

,index,sepal_length,sepal_width,petal_length,petal_width,target
144,144,6.7,3.3,5.7,2.5,2
145,145,6.7,3.0,5.2,2.3,2
146,146,6.3,2.5,5.0,1.9,2
147,147,6.5,3.0,5.2,2.0,2
148,148,6.2,3.4,5.4,2.3,2
149,149,5.9,3.0,5.1,1.8,2
150,1,2.0,3.0,4.0,5.0,6
151,1,2.0,3.0,4.0,5.0,6
152,1,2.0,3.0,4.0,5.0,688888
153,11,22.0,33.0,44.0,55.0,66


**Insert several lists at once**  
- If the value type in ANY of the lists does not match that in the associated row in the table, an error will return.
- This method only accepts lists whose length is equal to that of the number of columns in the target table, passing a longer or shorter list will return a custom error message. if ANY list in the values list is more or less values than the number of columns in the target table, an error message will return.

In [16]:
values_list = [[100,200,300,400,500,600], [111,222,333,444,555,777]]
my_db.insert_many('iris', values_list)

my_db.query('''SELECT * from iris''').tail(10)

,index,sepal_length,sepal_width,petal_length,petal_width,target
146,146,6.3,2.5,5.0,1.9,2
147,147,6.5,3.0,5.2,2.0,2
148,148,6.2,3.4,5.4,2.3,2
149,149,5.9,3.0,5.1,1.8,2
150,1,2.0,3.0,4.0,5.0,6
151,1,2.0,3.0,4.0,5.0,6
152,1,2.0,3.0,4.0,5.0,688888
153,11,22.0,33.0,44.0,55.0,66
154,100,200.0,300.0,400.0,500.0,600
155,111,222.0,333.0,444.0,555.0,777


**Attempting to insert a list that is too short**

In [17]:
my_db.insert_one('iris', [12,34,56,78,910])

Invalid value_list.  Length of value_list must be equal to number of columns.
            Table iris has 6 columns.


**Attempting to insert a list of lists where one list is too short**

In [18]:
values_list = [[101,202,303,404,505,606], [121,232,343,]]
my_db.insert_many('iris', values_list)

Invalid row in list_of_values_list. Length of rows in list_of_values_list
                must be equal to number of columns in target table.
                Table iris has 6 columns.


**Update table database/table using traditional SQL query.**
- Unlike most other methods in this class, `.update` utilizes a SQL query to update the database.

**_Notes_**
- _ This Method has not been tested as far as creating a new table on the current database._  
- _ You can potentially write a `SELECT` query in this method, however nothing will be returned._  
- *This method does not have the caveats of appropriate length like the `.insert_one` and `insert_many` methods.*

In [19]:
my_db.update('''INSERT INTO iris
                VALUES (111,212,313,414,515,616)''')
my_db.query('''SELECT * from iris''').tail(10)

,index,sepal_length,sepal_width,petal_length,petal_width,target
147,147,6.5,3.0,5.2,2.0,2
148,148,6.2,3.4,5.4,2.3,2
149,149,5.9,3.0,5.1,1.8,2
150,1,2.0,3.0,4.0,5.0,6
151,1,2.0,3.0,4.0,5.0,6
152,1,2.0,3.0,4.0,5.0,688888
153,11,22.0,33.0,44.0,55.0,66
154,100,200.0,300.0,400.0,500.0,600
155,111,222.0,333.0,444.0,555.0,777
156,111,212.0,313.0,414.0,515.0,616
